In [1]:
import os, sys
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

dir_list = ['..', "/home/hanlv/workspace/code/research/infodemic/LLM/autogen/create_prompt_bing_llm"]
for dir in dir_list:
    if not dir in sys.path:
        sys.path.append(dir)

from custom import CustomModelType, CustomTemplateType
import prompt_rag

from swift.llm import (
    get_model_tokenizer, get_template, inference,
)
from swift.utils import seed_everything

model_type = CustomModelType.solar_10_7b
template_type = CustomTemplateType.solar
print(f'template_type: {template_type}')  # template_type: qwen


kwargs = {}
kwargs['use_flash_attn'] = False  # 使用flash_attn

model, tokenizer = get_model_tokenizer(model_type, model_kwargs={'device_map': 'auto'}, **kwargs)

# 修改max_new_tokens
model.generation_config.max_new_tokens = 4096

template = get_template(template_type, tokenizer)
seed_everything(42)


2024-01-11 16:02:39,128 - modelscope - INFO - PyTorch version 2.1.2 Found.
2024-01-11 16:02:39,131 - modelscope - INFO - Loading ast index from /home/hanlv/.cache/modelscope/ast_indexer
2024-01-11 16:02:39,169 - modelscope - INFO - Loading done! Current index file version is 1.11.0, with md5 dc3cda2b56a86b0242cd145399a14071 and a total number of 953 components indexed
[INFO:swift] Setting torch_dtype: torch.float16
[INFO:swift] model_config: LlamaConfig {
  "_name_or_path": "/home/css/models/SOLAR-10.7B-Instruct-v1.0",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 14336,
  "max_position_embeddings": 4096,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 48,
  "num_key_value_heads": 8,
  "pad_token_id": 2,
  "pretraining_tp": 1,
  "rms_norm_eps": 1e-05,
  "rope_sc

template_type: solar


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

[INFO:swift] Global seed set to 42


42

In [6]:
import json

search_engine = "brave"

with open(f"/home/hanlv/workspace/data/machine_learning/dataset/research/misinformation_dataset/COVMIS-main/data/train_{search_engine}_search.json", "r") as f:
    data_search = json.load(f)

try:
    with open(f"/home/hanlv/workspace/data/machine_learning/dataset/research/misinformation_dataset/COVMIS-main/data/train_{search_engine}_search_llm.json", "r") as f:
        data_search_llm = json.load(f)
except:
    data_search_llm = [{
        "claim": i["claim"],
        "claimant": i["claimant"],
        "label": i["label"],
        "date": i["date"],
    } for i in data_search]
    with open(f"/home/hanlv/workspace/data/machine_learning/dataset/research/misinformation_dataset/COVMIS-main/data/train_{search_engine}_search_llm.json", "w") as f:
        json.dump(data_search_llm, f, indent=4)


In [7]:
K = 10
def get_id(claim):
    for i in range(len(data_search)):
        if claim.strip() in data_search[i]["claim"].strip():
            return i

# i = 0
i = get_id("COVID-19 Daily: Hospital Mortality Data, Drug Trial Raises Eyebrows")

claim = data_search[i]["claim"]
search_results = data_search[i][f"{search_engine}_search_results"]

prompt = prompt_rag.get_prompt_for_generating_prior_knowledge_v2(
    claim, data_search[i]["date"], search_engine, search_results, K=K, sort=False)
print(prompt)

response, history = inference(model, template, prompt)
print(response.strip())


Below is a CLAIM and some INFORMATION searched online. These pieces of INFORMATION are relevant to the CLAIM. This CLAIM and all INFORMATION include their respective publication dates and contents. To classify the CLAIM more accurately (if the content described by the CLAIM is correct, it will be classified as TRUE; if the content described by the CLAIM is incorrect, it will be classified as FALSE), please first expand on the given INFORMATION and provide a detailed summary of it. Then analyze, reason, and provide reasonable evidence to judge the correctness of the CLAIM based on the available information and your knowledge, and finally generate prior knowledge that helps classify the CLAIM.

CLAIM:
Publication date: 2020-04-30
Content: COVID-19 Daily: Hospital Mortality Data, Drug Trial Raises Eyebrows

INFORMATION:
Information 1:
Publication date: 2020-04-30
Title: COVID-19 Daily: Hospital Mortality Data, Drug Trial Raises Eyebrows
Content:
These are the coronavirus stories you need 

In [5]:
query = 'Hello?'
response, history = inference(
    model, template, query, 
    verbose=True, # 带有聊天模板
)
# print(f'query: {query}')
# print(f'response: {response}')


# # 流式输出对话模板
# inference(model, template, '第一个问题是什么', history, verbose=True, stream=True)


[PROMPT]### User:
Hello?

### Assistant:
[OUTPUT]Hello, how can I assist you today? Please feel free to ask me any questions or provide me with a task that I can help you with. I'm here to provide you with information, answer your queries, and make your life a little easier.</s>
